#### Top

# Chapter 2 Data Manipulation with Polars Using the Fuel Economy Dataset

* [2.0 Imports and Setup](#2.0-Imports-and-Setup)
* [2.1 Introduction](#2.1-Introduction)
* [2.2 Getting the Data](#2.2-Getting-the-Data)
  - [2.2.1 Notes on Categorical data](#2.2.1-Notes-on-Categorical-data)
* [2.3 Adding Columns](#2.3-Adding-Columns)
* [2.4 Simulating the Index](#2.4-Simulating-the-Index)
* [2.5 Removing Columns](#2.5-Removing-Columns)
* [2.6 Renaming Columns](#2.6-Renaming-Columns)
* [2.7 Left Joins](#2.7-Left-Joins)
* [2.8 Right Joins](#2.8-Right-Joins)
* [2.9 Inner Joins](#2.9-Inner-Joins)
* [2.10 Outer Joins](#2.10-Outer-Joins)
* [2.11 Semi Joins](#2.11-Semi-Joins)
* [2.12 Cross Joins](#2.12-Cross-Joins)
* [2.13 Anti Joins](#2.13-Anti-Joins)
* [2.14 Join Validation](#2.14-Join-Validation)
* [2.15 Speeding up Joins with Sorting](#2.15-Speeding-up-Joins-with-Sorting)
  - [2.15.1 Join OpenEI with autos data](#2.15.1-Join-OpenEI-with-autos-data)
  - [2.15.2 Joins Benchmarking](#2.15.2-Joins-Benchmarking)
* [2.16 Visualizing the Join](#2.16-Visualizing-the-Join)
* [2.17 Adding Rows](#2.17-Adding-Rows)
* [2.18 Reshape and Pivoting Data](#2.18-Reshape-and-Pivoting-Data)
* [2.19 Melted Data](#2.19-Melted-Data)
* [2.20 Finding Duplicates](#2.20-Finding-Duplicates)
* [2.21 Finding Missing Values](#2.21-Finding-Missing-Values)
* [2.22 Third-Party Libraries and Missing Values](#2.22-Third-Party-Libraries-and-Missing-Values)
* [2.23 Additional Column Selectors and Missing Values](#2.23-Additional-Column-Selectors-and-Missing-Values)
* [2.24 Map and Apply](#2.24-Map-and-Apply)



---
# 2.0 Imports and Setup
[back to Top](#Top)



*  polars.Config.set_tbl_formatting
* Adjust for better Table UX
* https://docs.pola.rs/py-polars/html/reference/api/polars.Config.set_tbl_formatting.html

In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import chardet
import pprint as pp
import hvplot.polars
hvplot.extension('matplotlib')

matplotlib_inline.backend_inline.set_matplotlib_formats("retina")
pd.options.mode.copy_on_write = True
print(pd.options.mode.copy_on_write)
pl.Config.set_verbose(True)
pl.show_versions()

def HR():
    print("-"*40)

@pl.Config(tbl_cols=-1, ascii_tables=True)
def tight_layout(df: pl.DataFrame, n=5) -> None:
    with pl.Config(tbl_cols=-1, fmt_str_lengths=n):
        print(df)

def detect_encoding(filename: str) -> str:
    """Return the most probable character encoding for a file."""
    with open(filename, "rb") as f:
        raw_data = f.read()
        print(raw_data)
        result = chardet.detect(raw_data)
        return result["encoding"]

True
--------Version info---------
Polars:              1.9.0
Index type:          UInt32
Platform:            macOS-12.7.6-x86_64-i386-64bit
Python:              3.11.5 (main, Jan 16 2024, 17:25:53) [Clang 14.0.0 (clang-1400.0.29.202)]

----Optional dependencies----
adbc_driver_manager  1.1.0
altair               5.4.0
cloudpickle          3.0.0
connectorx           0.3.3
deltalake            0.19.1
fastexcel            0.11.6
fsspec               2023.12.2
gevent               24.2.1
great_tables         0.10.0
matplotlib           3.9.2
nest_asyncio         1.6.0
numpy                2.0.2
openpyxl             3.1.5
pandas               2.2.2
pyarrow              17.0.0
pydantic             2.8.2
pyiceberg            0.6.1
sqlalchemy           2.0.32
torch                <not installed>
xlsx2csv             0.8.3
xlsxwriter           3.2.0


---
# 2.1 Introduction
[back to Top](#Top)

Goals:

* Manage and analyze data effectively
* Adding, removing, renaming columns
* Joining, concatenating, appending data
* Extract insights from data
* Handle a wide range of data manipulation tasks with Polars

---
# 2.2 Getting the Data

[back to Top](#Top)

In [2]:
path = 'data/vehicles.csv'
raw = pl.read_csv(path, null_values=['NA'])
print(raw.shape)
print(f"{raw.estimated_size(unit='mb'):.2f}MB")

(48202, 84)
29.34MB


avg line length: 434.78027
std. dev. line length: 23.885818
initial row estimate: 47850
no. of chunks: 4 processed by: 4 threads.


## 2.2.1 Notes on Categorical data

[back to Top](#Top)

https://docs.pola.rs/user-guide/concepts/data-types/categoricals/

*Enum vs Categorical*

Polars supports two different DataTypes for working with categorical data: Enum and Categorical. When the categories are known up front, use Enum. When you don't know the categories or they are not fixed, then use Categorical. In case your requirements change along the way you can always cast from one to the other.



*polars.StringCache*

Context manager for enabling and disabling the global string cache.

Categorical columns created under the same global string cache have the same underlying physical value when string values are equal. This allows the columns to be concatenated or used in a join operation, for example.

* https://docs.pola.rs/py-polars/html/reference/api/polars.StringCache.html


*CategoricalRemappingWarning*: 
Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance

In [3]:
@pl.StringCache()
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
           'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 
           'highway08', 'createdOn']
    return (
        df
        .select(pl.col(cols))
        .with_columns( 
            pl.col('year').cast(pl.Int16),
            pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
            pl.col(['displ', 'barrels08']).cast(pl.Float32),
            pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
            pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
            is_automatic=pl.col('trany')                    
               .str.contains('Automatic')
               .fill_null('Automatic'),
            num_gears=pl.col('trany')
                .str.extract(r'(\d+)')
                .cast(pl.UInt8)
                .fill_null(6)
        )
    )

autos = tweak_auto(raw)
print(type(autos))
print(autos.shape)
print(f"{autos.estimated_size(unit='mb'):.2f}MB")

<class 'polars.dataframe.frame.DataFrame'>
(48202, 15)
2.94MB


In [4]:
autos.head(1)

year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",14.167143,19,25,2013-01-01 00:00:00,"""false""",5


In [5]:
autos.flags

{'year': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'make': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'model': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'displ': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'cylinders': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'trany': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'drive': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'VClass': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'fuelType': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'barrels08': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'city08': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'highway08': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'createdOn': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'is_automatic': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'num_gears': {'SORTED_ASC': False, 'SORTED_DESC': False}}

In [6]:
{col:autos[col].is_sorted() for col in autos.columns}

{'year': False,
 'make': False,
 'model': False,
 'displ': False,
 'cylinders': False,
 'trany': False,
 'drive': False,
 'VClass': False,
 'fuelType': False,
 'barrels08': False,
 'city08': False,
 'highway08': False,
 'createdOn': False,
 'is_automatic': False,
 'num_gears': False}

---
# 2.3 Adding Columns
[back to Top](#Top)

* Cover the basic DataFrame operations for practical data analysis.
* Add, remove and rename columns.

*Example*
* Create a new column for fuel efficiency ratio of each vehicle.
* Divide highway MPG by the city MPG

In [7]:
# Ratio calculation
(
    autos
    .select(
        pl.col(['highway08', 'city08']),
        (pl.col('highway08') / pl.col('city08')).alias('mpg_ratio')
    )
)

highway08,city08,mpg_ratio
u8,u8,f64
25,19,1.315789
14,9,1.555556
33,23,1.434783
12,10,1.2
23,17,1.352941
…,…,…
26,19,1.368421
28,20,1.4
24,18,1.333333


Add the *mpg_ratio* column to the dataframe with `.with_columns`.
This preserves the existing dataframe columns and adds *mpg_ratio* as a new column at the end.

In [8]:
# Use .with_column to add this column
(
    autos
    .with_columns(
        (pl.col('highway08') / pl.col('city08')).alias('mpg_ratio')
    )
)

year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears,mpg_ratio
i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8,f64
1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",14.167143,19,25,2013-01-01 00:00:00,"""false""",5,1.315789
1985,"""Ferrari""","""Testarossa""",4.9,12,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",27.046364,9,14,2013-01-01 00:00:00,"""false""",5,1.555556
1985,"""Dodge""","""Charger""",2.2,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Subcompact Cars""","""Regular""",11.018888,23,33,2013-01-01 00:00:00,"""false""",5,1.434783
1985,"""Dodge""","""B150/B250 Wagon 2WD""",5.2,8,"""Automatic 3-spd""","""Rear-Wheel Drive""","""Vans""","""Regular""",27.046364,10,12,2013-01-01 00:00:00,"""true""",3,1.2
1993,"""Subaru""","""Legacy AWD Turbo""",2.2,4,"""Manual 5-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Premium""",15.658422,17,23,2013-01-01 00:00:00,"""false""",5,1.352941
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1993,"""Subaru""","""Legacy""",2.2,4,"""Automatic 4-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",13.523182,19,26,2013-01-01 00:00:00,"""true""",4,1.368421
1993,"""Subaru""","""Legacy""",2.2,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",12.935218,20,28,2013-01-01 00:00:00,"""false""",5,1.4
1993,"""Subaru""","""Legacy AWD""",2.2,4,"""Automatic 4-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Regular""",14.167143,18,24,2013-01-01 00:00:00,"""true""",4,1.333333


---
# 2.4 Simulating the Index
[back to Top](#Top)

* Polars does not have an index column like Pandas.
* If you need to rely on a numeric monotomic index, you can create a column to simulate the index with `.with_row_index`.
* https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.with_row_index.html
* Add a row index as the first column in the DataFrame.
* The resulting column does not have any special properties. It is a regular column of type UInt32 (or UInt64 in polars-u64-idx).

In [9]:
(
    autos
    .with_row_index('index')
    .filter(pl.col('index')<5)
)

dataframe filtered


index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
0,1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",14.167143,19,25,2013-01-01 00:00:00,"""false""",5
1,1985,"""Ferrari""","""Testarossa""",4.9,12,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",27.046364,9,14,2013-01-01 00:00:00,"""false""",5
2,1985,"""Dodge""","""Charger""",2.2,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Subcompact Cars""","""Regular""",11.018888,23,33,2013-01-01 00:00:00,"""false""",5
3,1985,"""Dodge""","""B150/B250 Wagon 2WD""",5.2,8,"""Automatic 3-spd""","""Rear-Wheel Drive""","""Vans""","""Regular""",27.046364,10,12,2013-01-01 00:00:00,"""true""",3
4,1993,"""Subaru""","""Legacy AWD Turbo""",2.2,4,"""Manual 5-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Premium""",15.658422,17,23,2013-01-01 00:00:00,"""false""",5


In [10]:
# Add only if does not exist
if not "index" in autos.columns:
    print("Adding index")
    autos = autos.with_row_index('index')
else:
    print("index exists")

# Check end of data
autos.tail(2)

Adding index


index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
48200,1993,"""Subaru""","""Legacy AWD""",2.2,4,"""Manual 5-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Regular""",14.167143,18,24,2013-01-01 00:00:00,"""false""",5
48201,1993,"""Subaru""","""Legacy AWD Turbo""",2.2,4,"""Automatic 4-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Premium""",16.528334,16,21,2013-01-01 00:00:00,"""true""",4


---
# 2.5 Removing Columns
[back to Top](#Top)

Three basic ways to remove columns from a DataFrame:
  1. `.drop()`
  2. `.select()`
  3. `.exclude`


In [11]:
print(
    autos
    .drop('createdOn')
    .columns
)

['index', 'year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'is_automatic', 'num_gears']


In [12]:
final_cols = ['index', 'year', 'make', 'model', 'displ', 'cylinders', 'trany', 
              'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 
              'highway08', 'is_automatic', 'num_gears']

In [13]:
(
    autos
    .select(final_cols)
).tail(2)

index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,str,u8
48200,1993,"""Subaru""","""Legacy AWD""",2.2,4,"""Manual 5-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Regular""",14.167143,18,24,"""false""",5
48201,1993,"""Subaru""","""Legacy AWD Turbo""",2.2,4,"""Automatic 4-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Premium""",16.528334,16,21,"""true""",4


* Polars also has an `.exclude` method on a column expression.
* This is probably more flexible than the `.drop` context because you can perform further operations on the columns that are not excluded.
* Being positive about what columns you want to preserve (rather than indicating the columns to drop) can aid the Polars query planner operating in lazy mode.

In [14]:
(
    autos
    .select(
        pl.all()
        .exclude(['createdOn', 'barrels08'])
        .name.suffix('_auto')
    )
    .head(2)
)

index_auto,year_auto,make_auto,model_auto,displ_auto,cylinders_auto,trany_auto,drive_auto,VClass_auto,fuelType_auto,city08_auto,highway08_auto,is_automatic_auto,num_gears_auto
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,u8,u8,str,u8
0,1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",19,25,"""false""",5
1,1985,"""Ferrari""","""Testarossa""",4.9,12,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",9,14,"""false""",5


---
# 2.6 Renaming Columns
[back to top](#Top)

* Unlike Pandas, Polars does not allow duplicate column names.
* We can rename a column using the `.alias` method or keyword argument. 

*Example*
* Rename city08 to city_mpg, and highway08 to highway_mpg. 

In [15]:
final_cols = ['year', 'make', 'model', 'city_mpg', 'highway_mpg']

(
    autos
    .with_columns(
        pl.col('city08').alias('city_mpg'), # style 1
        highway_mpg=pl.col('highway08'), # style 2
    )
    .select(final_cols)
)

year,make,model,city_mpg,highway_mpg
i16,cat,cat,u8,u8
1985,"""Alfa Romeo""","""Spider Veloce 2000""",19,25
1985,"""Ferrari""","""Testarossa""",9,14
1985,"""Dodge""","""Charger""",23,33
1985,"""Dodge""","""B150/B250 Wagon 2WD""",10,12
1993,"""Subaru""","""Legacy AWD Turbo""",17,23
…,…,…,…,…
1993,"""Subaru""","""Legacy""",19,26
1993,"""Subaru""","""Legacy""",20,28
1993,"""Subaru""","""Legacy AWD""",18,24



There is also a dataframe method `.rename` that accepts a dictionary mapping for renaming columns.

In [16]:
final_cols = ['year', 'make', 'model', 'city_mpg', 'highway_mpg']

(
    autos
    .rename({
        'city08': 'city_mpg',
        'highway08': 'highway_mpg'
    })
    .select(final_cols)
)

year,make,model,city_mpg,highway_mpg
i16,cat,cat,u8,u8
1985,"""Alfa Romeo""","""Spider Veloce 2000""",19,25
1985,"""Ferrari""","""Testarossa""",9,14
1985,"""Dodge""","""Charger""",23,33
1985,"""Dodge""","""B150/B250 Wagon 2WD""",10,12
1993,"""Subaru""","""Legacy AWD Turbo""",17,23
…,…,…,…,…
1993,"""Subaru""","""Legacy""",19,26
1993,"""Subaru""","""Legacy""",20,28
1993,"""Subaru""","""Legacy AWD""",18,24


---
# 2.7 Left Joins
[back to Top](#Top)

Joining dataframes is a typical operation in data analysis. It allows us to combine data from multiple sources into a single dataframe:

* Inner Joins
* Outer Joins
* Left Joins
* Right Joins
* Cross Joins
* Anti Joins

Start with *left joins*

* **Preserve all the rows from the left dataframe** (possibly duplicating them).
* Adds the columns from the right dataframe, where appropriate.
* If there are rows in the left dataframe that don't have a match in the right dataframe, the columns from the RD will be filled with null values.
* If there are rows in the left dataframe that match multiple rows in the right dataframe, you will get a row for each match.

In [17]:
# dataframe 1
trucks = pl.DataFrame({
    'make': ['Ford', 'Tesla', 'Chevy', 'Custom', 'Ford'],
    'model': ['F150', 'Cybertruck', 'Silverado', 'HotRod', 'F250'], 
    'year': [2018, 2024, 2019, 1967, 2017],
    'city_mpg': [19, None, 17, 12, 18],
}).sort(by='make')

print(trucks)
trucks.flags

shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+------------+------+----------+


{'make': {'SORTED_ASC': True, 'SORTED_DESC': False},
 'model': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'year': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'city_mpg': {'SORTED_ASC': False, 'SORTED_DESC': False}}

In [18]:
# dataframe 2
manufacturer = pl.DataFrame({
    'name': ['Ford', 'Tesla', 'Chevy', 'Toyota'],
    'country': ['USA', 'USA', 'USA', 'Japan'],
    'founded': [1903, 2003, 1911, 1937],
    'employees': [199_000, 48_000, 225_000, 370_000],
    'vehicles': [80, 3, 45, 30],
}).sort(by='name')

print(manufacturer)
manufacturer.flags

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+


{'name': {'SORTED_ASC': True, 'SORTED_DESC': False},
 'country': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'founded': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'employees': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'vehicles': {'SORTED_ASC': False, 'SORTED_DESC': False}}

In [19]:
print(manufacturer)
print(trucks)

(
    manufacturer
    .join(
        trucks, 
        how='left', 
        left_on='name',
        right_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
LEFT join dataframes finished


name,country,founded,employees,vehicles,model,year,city_mpg
str,str,i64,i64,i64,str,i64,i64
"""Chevy""","""USA""",1911,225000,45,"""Silverado""",2019,17
"""Ford""","""USA""",1903,199000,80,"""F150""",2018,19
"""Ford""","""USA""",1903,199000,80,"""F250""",2017,18
"""Tesla""","""USA""",2003,48000,3,"""Cybertruck""",2024,null
"""Toyota""","""Japan""",1937,370000,30,null,null,null


---
# 2.8 Right Joins
[back to Top](#Top)

* Polars doesn't support how='right' in `.join()`
* Instead, just use `how=left`, and swap the dataframes, `left_on`, and `right_on`.
* This keeps all the rows from the right dataframe and adds the columns from the left dataframe. 
* If there are rows in the right dataframe that don't have a match in the left dataframe, the columns from the left dataframe will be filled with null values.
* Place the data you want to keep on the left.

In [20]:
print(manufacturer)
print(trucks)

(
    trucks
    .join(
        manufacturer,
        how='left',
        right_on='name',
        left_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
LEFT join dataframes finished


make,model,year,city_mpg,country,founded,employees,vehicles
str,str,i64,i64,str,i64,i64,i64
"""Chevy""","""Silverado""",2019,17,"""USA""",1911,225000,45
"""Custom""","""HotRod""",1967,12,null,null,null,null
"""Ford""","""F150""",2018,19,"""USA""",1903,199000,80
"""Ford""","""F250""",2017,18,"""USA""",1903,199000,80
"""Tesla""","""Cybertruck""",2024,null,"""USA""",2003,48000,3


---
# 2.9 Inner Joins
[back to Top](#Top)

* An inner join keeps only rows that have a **match in both DataFrames**.
* **This is the default join type in Polars.**
* You use this when you want to **preserve the data that is common** to both DataFrames.
* If multiple rows match, it will include each match.
* This join will often simplify the data analysis because you don't have to worry about null vlaues that result from a join.

In [21]:
print(manufacturer)
print(trucks)

(
    manufacturer
    .join(
        trucks,
        how='inner',
        left_on='name',
        right_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
INNER join dataframes finished


name,country,founded,employees,vehicles,model,year,city_mpg
str,str,i64,i64,i64,str,i64,i64
"""Chevy""","""USA""",1911,225000,45,"""Silverado""",2019,17
"""Ford""","""USA""",1903,199000,80,"""F150""",2018,19
"""Ford""","""USA""",1903,199000,80,"""F250""",2017,18
"""Tesla""","""USA""",2003,48000,3,"""Cybertruck""",2024,null


---
# 2.10 Outer Joins
[back to Top](#Top)

* An outer join keeps all of the rows from both DataFrames. 
* If there is a match in the other dataframe, the columns from the other dataframe are included.
* If there is no match in the other dataframe, the columns from the other dataframe are filled with null values. 
* Use an outer join to keep all of the data from both dataframes.

In [22]:
print(manufacturer)
print(trucks)

(
    manufacturer.join(
        trucks,
        how='full', # outer is deprecated
        left_on='name',
        right_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
FULL join dataframes finished


name,country,founded,employees,vehicles,make,model,year,city_mpg
str,str,i64,i64,i64,str,str,i64,i64
"""Chevy""","""USA""",1911,225000,45,"""Chevy""","""Silverado""",2019,17
null,null,null,null,null,"""Custom""","""HotRod""",1967,12
"""Ford""","""USA""",1903,199000,80,"""Ford""","""F150""",2018,19
"""Ford""","""USA""",1903,199000,80,"""Ford""","""F250""",2017,18
"""Tesla""","""USA""",2003,48000,3,"""Tesla""","""Cybertruck""",2024,null
"""Toyota""","""Japan""",1937,370000,30,null,null,null,null


---
# 2.11 Semi Joins
[back to Top](#Top)

* A *semi-join* keeps only the rows from the left dataframe that have a match with the right dataframe.
* It does not add any columns from the right dataframe.
* This is useful when you want to filter the rows in the left dataframe based on rows in the right dataframe.
* It differs from a left join because it **doesn't add any columns from the right dataframe.**
* It differs from an inner join because it **doesn't keep any rows that don't have a match in the right dataframe.**

If we want to keep all the rows from *manufacturer* that have a match in *trucks*, we can use a semi-join. This will keep the Ford, Tesla and Checy rows from manufacturer. It drops the Toyota row.

In [23]:
print(manufacturer)
print(trucks)

(
    manufacturer.join(
        trucks, 
        how='semi',
        left_on='name',
        right_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
SEMI join dataframes finished


name,country,founded,employees,vehicles
str,str,i64,i64,i64
"""Chevy""","""USA""",1911,225000,45
"""Ford""","""USA""",1903,199000,80
"""Tesla""","""USA""",2003,48000,3


---
# 2.12 Cross Joins
[back to Top](#Top)

* Doesn't match up the rows based on column values.
* A cross-join combines every row from the left dataframe with every row from the right dataframe.
* This is also known as a **Cartesian product**.
* This is rarely used because it can result in a very large dataframe.

In [24]:
sizes = pl.DataFrame({
    'size': ['small', 'medium', 'large']
})
colors = pl.DataFrame({
    'color': ['red', 'green']
})

print(sizes)
print(colors)

# make a combination of every row on the left and right
(
    sizes.join(colors, how='cross')
)

shape: (3, 1)
+--------+
| size   |
| ---    |
| str    |
+========+
| small  |
| medium |
| large  |
+--------+
shape: (2, 1)
+-------+
| color |
| ---   |
| str   |
+=======+
| red   |
| green |
+-------+


join parallel: true
CROSS join dataframes finished


size,color
str,str
"""small""","""red"""
"""small""","""green"""
"""medium""","""red"""
"""medium""","""green"""
"""large""","""red"""
"""large""","""green"""


In [25]:
(
    manufacturer.join(
        trucks, 
        how='cross'
    )
)

join parallel: true
CROSS join dataframes finished


name,country,founded,employees,vehicles,make,model,year,city_mpg
str,str,i64,i64,i64,str,str,i64,i64
"""Chevy""","""USA""",1911,225000,45,"""Chevy""","""Silverado""",2019,17
"""Chevy""","""USA""",1911,225000,45,"""Custom""","""HotRod""",1967,12
"""Chevy""","""USA""",1911,225000,45,"""Ford""","""F150""",2018,19
"""Chevy""","""USA""",1911,225000,45,"""Ford""","""F250""",2017,18
"""Chevy""","""USA""",1911,225000,45,"""Tesla""","""Cybertruck""",2024,null
…,…,…,…,…,…,…,…,…
"""Toyota""","""Japan""",1937,370000,30,"""Chevy""","""Silverado""",2019,17
"""Toyota""","""Japan""",1937,370000,30,"""Custom""","""HotRod""",1967,12
"""Toyota""","""Japan""",1937,370000,30,"""Ford""","""F150""",2018,19


---
# 2.13 Anti-Joins
[back to Top](#Top)

* Keep all the rows from the left dataframe that don't have a match in the right dataframe.
* Does not keep any rows that have a match in the right dataframe.
* Useful when you want to find the rows in one dataframe but not in the other.
* Can be very useful for debugging.

An anti-join can be used to determine **which manufacturers are not in the trucks dataframe.**

In [26]:
print(manufacturer)
print(trucks)

(
    manufacturer.join(
        trucks,
        how='anti',
        left_on='name',
        right_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
ANTI join dataframes finished


name,country,founded,employees,vehicles
str,str,i64,i64,i64
"""Toyota""","""Japan""",1937,370000,30


---
If we want to know **which trucks are not in the manufacturers dataframe**, we can put the manufacturers dataframe on the right side of the anti-join.

In [27]:
print(manufacturer)
print(trucks)

(
    trucks.join(
        manufacturer,
        how='anti',
        right_on='name',
        left_on='make'
    )
)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
ANTI join dataframes finished


make,model,year,city_mpg
str,str,i64,i64
"""Custom""","""HotRod""",1967,12


---
# 2.14 Join Validation
[back to Top](#Top)

* When you join two dataframes, you want to understand if the join is a one-to-one join, a one-to-many join, or a many-to-many join.
* This will help you understand the structure of the data and how to use it in your analysis.
* It can also prevent errors.
* For example, if you run an online store and allow customers to have multiple shipping addresses, you want to ensure that a shipment only goes to one address.
* If you had an orders table and a shipping table, you would want to ensure that the join is a one-to-one join.

In [28]:
print(manufacturer)
print(trucks)

try:
    print(
        manufacturer.join(
            trucks,
            left_on='name',
            right_on='make',
            validate='1:1'
        )
    )
except Exception as e:
    print(e)

shape: (4, 5)
+--------+---------+---------+-----------+----------+
| name   | country | founded | employees | vehicles |
| ---    | ---     | ---     | ---       | ---      |
| str    | str     | i64     | i64       | i64      |
+===================================================+
| Chevy  | USA     | 1911    | 225000    | 45       |
| Ford   | USA     | 1903    | 199000    | 80       |
| Tesla  | USA     | 2003    | 48000     | 3        |
| Toyota | Japan   | 1937    | 370000    | 30       |
+--------+---------+---------+-----------+----------+
shape: (5, 4)
+--------+------------+------+----------+
| make   | model      | year | city_mpg |
| ---    | ---        | ---  | ---      |
| str    | str        | i64  | i64      |
+=======================================+
| Chevy  | Silverado  | 2019 | 17       |
| Custom | HotRod     | 1967 | 12       |
| Ford   | F150       | 2018 | 19       |
| Ford   | F250       | 2017 | 18       |
| Tesla  | Cybertruck | 2024 | null     |
+--------+--

join parallel: true
INNER join dataframes finished


In [29]:
# try many-to-one validation
try:
    print(
        manufacturer.join(
            trucks,
            left_on='name',
            right_on='make',
            validate='m:1'
        )
    )
except Exception as e:
    print(e)

join keys did not fulfill m:1 validation


join parallel: true
INNER join dataframes finished


In [30]:
# try  1:many validation
try:
    print(
        manufacturer.join(
            trucks,
            left_on='name',
            right_on='make',
            validate='1:m'
        )
    )
except Exception as e:
    print(e)

shape: (4, 8)
+-------+---------+---------+-----------+----------+------------+------+----------+
| name  | country | founded | employees | vehicles | model      | year | city_mpg |
| ---   | ---     | ---     | ---       | ---      | ---        | ---  | ---      |
| str   | str     | i64     | i64       | i64      | str        | i64  | i64      |
+=================================================================================+
| Chevy | USA     | 1911    | 225000    | 45       | Silverado  | 2019 | 17       |
| Ford  | USA     | 1903    | 199000    | 80       | F150       | 2018 | 19       |
| Ford  | USA     | 1903    | 199000    | 80       | F250       | 2017 | 18       |
| Tesla | USA     | 2003    | 48000     | 3        | Cybertruck | 2024 | null     |
+-------+---------+---------+-----------+----------+------------+------+----------+


join parallel: true
INNER join dataframes finished


---
# 2.15 Speeding up Joins with Sorting
[back to Top](#Top)

* Polars has a query engine and can be smart about joins.
* Polars can use a faster algorithm to join data if your data is sorted.
* Here, use the Open Energy Information (OpenEI) dataset.


In [31]:
url = 'https://data.openei.org/files/907/2016cityandcountylightdutyvehicleinventory.xlsb'
url_file = './data/2016cityandcountylightdutyvehicleinventory.xlsb'

!wget -P ./data -nc $url

--2024-10-25 10:54:35--  https://data.openei.org/files/907/2016cityandcountylightdutyvehicleinventory.xlsb
data.openei.org (data.openei.org) をDNSに問いあわせています... 35.164.34.232, 52.43.166.203, 52.11.187.133
data.openei.org (data.openei.org)|35.164.34.232|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 62605647 (60M) [application/octet-stream]
`./data/2016cityandcountylightdutyvehicleinventory.xlsb' に保存中

2016cityandcountyli 100%[===================>]  59.71M  1.36MB/s 時間 29s        

2024-10-25 10:55:05 (2.05 MB/s) - `./data/2016cityandcountylightdutyvehicleinventory.xlsb' へ保存完了 [62605647/62605647]



In [32]:
inv_raw = pl.read_excel(
    url_file,
    engine='calamine',
    read_options=dict(header_row=1),
    sheet_name='City'
)
inv_raw.shape

dataframe filtered


(307828, 51)

In [33]:
{col:inv_raw[col].is_sorted() for col in inv_raw.columns}

{'state_abbr': False,
 'gisjoin': False,
 'city_id': True,
 'city_name': False,
 'fuel_type_org': False,
 'fuel_type': False,
 'class': False,
 'before 1980': False,
 '1980-99': False,
 '1990-99': False,
 '2000': False,
 '2001': False,
 '2002': False,
 '2003': False,
 '2004': False,
 '2005': False,
 '2006': False,
 '2007': False,
 '2008': False,
 '2009': False,
 '2010': False,
 '2011': False,
 '2012': False,
 '2013': False,
 '2014': False,
 '2015': False,
 '2016': False,
 '2017': False,
 '2018': True,
 'before 1980_1': False,
 '1980-99_1': False,
 '1990-99_1': False,
 '2000_1': False,
 '2001_1': False,
 '2002_1': False,
 '2003_1': False,
 '2004_1': False,
 '2005_1': False,
 '2006_1': False,
 '2007_1': False,
 '2008_1': False,
 '2009_1': False,
 '2010_1': False,
 '2011_1': False,
 '2012_1': False,
 '2013_1': False,
 '2014_1': False,
 '2015_1': False,
 '2016_1': False,
 '2017_1': False,
 '2018_1': True}

In [34]:
inv_raw

state_abbr,gisjoin,city_id,city_name,fuel_type_org,fuel_type,class,before 1980,1980-99,1990-99,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,before 1980_1,1980-99_1,1990-99_1,2000_1,2001_1,2002_1,2003_1,2004_1,2005_1,2006_1,2007_1,2008_1,2009_1,2010_1,2011_1,2012_1,2013_1,2014_1,2015_1,2016_1,2017_1,2018_1
str,str,i64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null
"""AL""","""G01000124""",100124,"""Abbeville""","""BI""","""Other/Unknown""","""Car""",null,null,null,null,0.000135,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.000006,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Car""",null,0.000676,null,0.00027,null,null,0.000135,null,null,null,null,null,null,null,0.000135,null,null,0.000135,0.000135,null,null,null,null,0,null,0.0,null,null,0.0,null,null,null,null,null,null,null,0.000006,null,null,0.000005,0.0,null,null,null
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Truck""",null,0.000946,0.005003,0.001622,0.001893,0.001487,0.001622,0.001758,0.002299,0.002028,0.001758,0.001082,0.000541,0.000676,0.001217,0.001217,0.000946,0.001352,0.001893,0.002028,null,null,null,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00009,null,null
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Unknown""",0.000541,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""AL""","""G01000124""",100124,"""Abbeville""","""ELECTRIC VEHICLE""","""Electric vehicle""","""Car""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.000135,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.000001,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Car""",null,0.020339,0.079661,0.022034,0.016949,0.013559,0.022034,0.023729,0.010169,0.00678,0.022034,0.022034,0.013559,0.008475,0.018644,0.013559,0.013559,0.020339,0.016949,0.00678,null,null,null,0,0,0.001013,0.000734,0.000673,0.000965,0.001225,0.00045,0.00029,0.001034,0.001102,0.000904,0.000456,0.000896,0.000549,0.000581,0.000887,0.000772,0.0,null,null
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Truck""",null,0.040678,0.115254,0.022034,0.013559,0.016949,0.018644,0.00678,0.013559,0.011864,0.015254,0.00678,0.001695,0.00339,0.001695,0.008475,0.005085,0.010169,0.016949,0.00339,null,null,null,0,0,0.001425,0.00085,0.001049,0.001172,0.000399,0.000871,0.000744,0.000885,0.0,0.0,0.000195,0.000113,0.000499,0.000299,0.000566,0.000936,0.000188,null,null
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Unknown""",0.023729,null,0.001695,null,null,null,null,null,0.001695,null,null,null,null,null,null,null,null,null,null,null,null,null,0,null,0,null,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null


In [35]:
(
    inv_raw
    .select([
        'state_abbr', 'gisjoin', 'city_id', 'city_name',
        'fuel_type_org', 'fuel_type', 'class', '2000', '2001'
    ])
)

state_abbr,gisjoin,city_id,city_name,fuel_type_org,fuel_type,class,2000,2001
str,str,i64,str,str,str,str,f64,f64
"""AL""","""G01000124""",100124,"""Abbeville""","""BI""","""Other/Unknown""","""Car""",null,0.000135
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Car""",0.00027,null
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Truck""",0.001622,0.001893
"""AL""","""G01000124""",100124,"""Abbeville""","""DIES""","""Diesel vehicle""","""Unknown""",null,null
"""AL""","""G01000124""",100124,"""Abbeville""","""ELECTRIC VEHICLE""","""Electric vehicle""","""Car""",null,null
…,…,…,…,…,…,…,…,…
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Car""",0.022034,0.016949
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Truck""",0.022034,0.013559
"""WY""","""G56086665""",5686665,"""Yoder""","""GAS""","""Gasoline vehicle""","""Unknown""",null,null


---
Aggregate the city mileage for each vehicle type, and then join it with the autos dataframe.

In [36]:
years = [str(i) for i in range(2000,2019)]
years

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018']

In [37]:
# Test
(
    inv_raw
    .select(
        [pl.col(year).cast(pl.Int16) for year in years]
    )
    # .unpivot(
    #     variable_name='year',
    #     on=years,
    #     index=['state_abbr', 'gisjoin', 'city_id', 'city_name', 'fuel_type_org', 'fuel_type', 'class']
    # ) 
)

2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16
null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,null,null,0,null,null,null,null,null,null,null,0,null,null,0,0,null,null,null
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null
null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null


In [38]:
inv_yr = (
    inv_raw
    .with_columns(
        [pl.col(year).cast(pl.Int16) for year in years]
    )
    .unpivot(
        variable_name='year',
        on=years,
        index=['state_abbr', 'gisjoin', 'city_id', 'city_name', 'fuel_type_org', 'fuel_type', 'class']
    )
    .with_columns(
        year=pl.col('year').cast(pl.Int16),
        percent=(pl.col('value')*100).cast(pl.Float32)
    )
    .select(
        'year', 'percent', 'state_abbr', 'city_name', 'fuel_type_org', 'fuel_type', 'class',
    )
)

inv_yr

year,percent,state_abbr,city_name,fuel_type_org,fuel_type,class
i16,f32,str,str,str,str,str
2000,null,"""AL""","""Abbeville""","""BI""","""Other/Unknown""","""Car"""
2000,0.0,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Car"""
2000,0.0,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Truck"""
2000,null,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Unknown"""
2000,null,"""AL""","""Abbeville""","""ELECTRIC VEHICLE""","""Electric vehicle""","""Car"""
…,…,…,…,…,…,…
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Car"""
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Truck"""
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Unknown"""


In [39]:
# Updated version using unpivot
inv_yr = (
    inv_raw
    .with_columns(
        [pl.col(year).cast(pl.Int16) for year in years]
    )
    .unpivot(
        variable_name='year',
        on=years,
        index=['state_abbr', 'gisjoin', 'city_id', 'city_name', 'fuel_type_org', 'fuel_type', 'class']
    )
    .with_columns(
        year=pl.col('year').cast(pl.Int16),
        percent=(pl.col('value')*100).cast(pl.Float32)
    )
    .select(
        'year', 'percent', 'state_abbr', 'city_name', 'fuel_type_org', 'fuel_type', 'class',
        # year=pl.col('year').cast(pl.Int16),
        # percent=(pl.col('value')*100).cast(pl.Float32)
    )
)

inv_yr

year,percent,state_abbr,city_name,fuel_type_org,fuel_type,class
i16,f32,str,str,str,str,str
2000,null,"""AL""","""Abbeville""","""BI""","""Other/Unknown""","""Car"""
2000,0.0,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Car"""
2000,0.0,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Truck"""
2000,null,"""AL""","""Abbeville""","""DIES""","""Diesel vehicle""","""Unknown"""
2000,null,"""AL""","""Abbeville""","""ELECTRIC VEHICLE""","""Electric vehicle""","""Car"""
…,…,…,…,…,…,…
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Car"""
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Truck"""
2018,null,"""WY""","""Yoder""","""GAS""","""Gasoline vehicle""","""Unknown"""


---
## 2.15.1 Join OpenEI with autos data
[back to Top](#Top)

* We can join `inv_yr` with the `autos` data.
* We want aggregated mileage data: mean city milage for each year, vehicle type and fuel type.
* Convert VClass to a string
* Create a new column called simple_class that generalizes VClass
* Cast fuel_type to a string
* Then join by year, simple_class and fuel_type.
* Then aggregate the city mileage with the mean function.

We are working with melting and aggregating here.

In [40]:
# Example
df = pl.DataFrame({"foo": [1, 3, 4], "bar": [3, 4, 0]})
print(df)

#print(df.with_columns(pl.when(pl.col("foo") > 2).then(1).otherwise(-1).alias("val")))
print(df.with_columns(VALUE_MINUS1_OR_1 = pl
                      .when(pl.col("foo") > 2).then(1).otherwise(-1)))

shape: (3, 2)
+-----+-----+
| foo | bar |
| --- | --- |
| i64 | i64 |
+===========+
| 1   | 3   |
| 3   | 4   |
| 4   | 0   |
+-----+-----+
shape: (3, 3)
+-----+-----+-------------------+
| foo | bar | VALUE_MINUS1_OR_1 |
| --- | --- | ---               |
| i64 | i64 | i32               |
+===============================+
| 1   | 3   | -1                |
| 3   | 4   | 1                 |
| 4   | 0   | 1                 |
+-----+-----+-------------------+


In [41]:
# Make a simple category
gas_mapping = {
    'Diesel': 'Diesel vehicle',
    'Regular': 'Gasoline vehicle',
    'Premium': 'Gasoline vehicle',
    'Midgrade': 'Gasoline vehicle',
    'Gasoline or E85': 'Flex fuel vehicle',
    'Premium or E85': 'Flex fuel vehicle',
    'Premium Gas or Electricity': 'Plug-in hybrid electric vehicle',
    'Regular Gas or Electricity': 'Plug-in hybrid electric vehicle',
    'Premium and Electricity': 'Hybrid electric vehicle',
    'Regular Gas and Electricity': 'Hybrid electric vehicle',
    'Electricity': 'Electric vehicle',
    'Gasoline or natural gas': 'Other/Unknown',
    'Gasoline or propane': 'Other/Unknown',
    'CNG': 'Other/Unknown',
}

agg_yr = (
    autos
    .with_columns(VClass=pl.col('VClass').cast(pl.String))
    .with_columns(
        simple_class=pl
        .when(pl.col('VClass').str.to_lowercase().str.contains('car'))
            .then(pl.lit('Car'))
        .when(pl.col('VClass').str.to_lowercase().str.contains('truck'))
            .then(pl.lit('Truck'))
        .otherwise(pl.lit('Other')),
        
        fuel_type=pl.col('fuelType')
            .cast(pl.String)
            #.replace(gas_mapping, default='Missing') # deprecated
            .replace_strict(gas_mapping, default='Missing')
    )
    .group_by(['year', 'simple_class', 'fuel_type'])
    .agg(mean_mpg = pl.col('city08').mean())
)
print(inv_yr.shape)
print(agg_yr.shape)
print(inv_yr)
print(agg_yr)

(5848732, 7)
(445, 4)
shape: (5_848_732, 7)
+------+---------+------------+-----------+------------------+---------------------------+---------+
| year | percent | state_abbr | city_name | fuel_type_org    | fuel_type                 | class   |
| ---  | ---     | ---        | ---       | ---              | ---                       | ---     |
| i16  | f32     | str        | str       | str              | str                       | str     |
+==================================================================================================+
| 2000 | null    | AL         | Abbeville | BI               | Other/Unknown             | Car     |
| 2000 | 0.0     | AL         | Abbeville | DIES             | Diesel vehicle            | Car     |
| 2000 | 0.0     | AL         | Abbeville | DIES             | Diesel vehicle            | Truck   |
| 2000 | null    | AL         | Abbeville | DIES             | Diesel vehicle            | Unknown |
| 2000 | null    | AL         | Abbeville | ELE

keys/aggregates are not partitionable: running default HASH AGGREGATION


---
Join these two datasets by year, vehicle class, and fuel type.


In [42]:
(
    agg_yr
    .join(
        inv_yr,
        left_on=['year', 'simple_class', 'fuel_type'],
        right_on=['year', 'class', 'fuel_type']
    )
)

join parallel: true
INNER join triggered a rechunk of the left DataFrame: 4 columns are affected
INNER join dataframes finished


year,simple_class,fuel_type,mean_mpg,percent,state_abbr,city_name,fuel_type_org
i16,str,str,f64,f32,str,str,str
2000,"""Car""","""Other/Unknown""",17.6,null,"""AL""","""Abbeville""","""BI"""
2000,"""Car""","""Diesel vehicle""",32.0,0.0,"""AL""","""Abbeville""","""DIES"""
2000,"""Car""","""Flex fuel vehicle""",17.0,0.0,"""AL""","""Abbeville""","""FFV"""
2000,"""Truck""","""Flex fuel vehicle""",15.0,0.0,"""AL""","""Abbeville""","""FFV"""
2000,"""Car""","""Gasoline vehicle""",18.129032,0.0,"""AL""","""Abbeville""","""GAS"""
…,…,…,…,…,…,…,…
2018,"""Car""","""Flex fuel vehicle""",18.875,null,"""WY""","""Yoder""","""FFV"""
2018,"""Truck""","""Flex fuel vehicle""",16.565217,null,"""WY""","""Yoder""","""FFV"""
2018,"""Car""","""Gasoline vehicle""",22.004785,null,"""WY""","""Yoder""","""GAS"""


---
Do some simple quick benchmarks to see if sorting or using categoricals has an impact on joining.

`inv_yr` has + 5 million rows.

In [43]:
agg_yr_shuf = agg_yr.sample(len(agg_yr), with_replacement=False, seed=42)
print(agg_yr_shuf.shape)

inv_yr_shuf = inv_yr.sample(len(inv_yr), with_replacement=False, seed=42)
print(inv_yr_shuf.shape)

(445, 4)
(5848732, 7)


In [44]:
agg_yr_shuf.flags

{'year': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'simple_class': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'fuel_type': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'mean_mpg': {'SORTED_ASC': False, 'SORTED_DESC': False}}

In [45]:
inv_yr_shuf.flags

{'year': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'percent': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'state_abbr': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'city_name': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'fuel_type_org': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'fuel_type': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'class': {'SORTED_ASC': False, 'SORTED_DESC': False}}

---
## 2.15.2 Joins Benchmarking
[back to Top](#Top)

In [46]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    
    # baseline time
    (
        agg_yr_shuf
        .join(
            inv_yr_shuf,
            left_on=['year', 'simple_class', 'fuel_type'],
            right_on=['year', 'class', 'fuel_type']
        )
    )

timing_test()

616 ms ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
Sort each dataframe by *year* and run it again.

In [47]:
# sort by year
agg_yr_sort = agg_yr.sort('year')
inv_yr_sort = inv_yr.sort('year')

In [48]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    # baseline time
    (
        agg_yr_sort
        .join(
            inv_yr_sort,
            left_on=['year', 'simple_class', 'fuel_type'],
            right_on=['year', 'class', 'fuel_type']
        )
    )

timing_test()

545 ms ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
* There is not much of a difference here.

* This time, try sorting all of the joining columns.

In [49]:
# sort by year and class
agg_yr_sort2 = agg_yr.sort('year', 'simple_class', 'fuel_type')
inv_yr_sort2 = inv_yr.sort('year', 'class', 'fuel_type')

In [50]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    (
        agg_yr_sort2
        .join(
            inv_yr_sort2,
            left_on=['year', 'simple_class', 'fuel_type'],
            right_on=['year', 'class', 'fuel_type']
        )
    )

timing_test()

700 ms ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
Try converting the strings to categorical values.

We need to use the `pl.StringCache` context manager and convert all of the columns to categories inside of the same context.

When you are working with multiple dataframes that have categoricals with the same values, you can use the StringCache to ensure that the categories are the same across the dataframes. This can help with joins and aggregations.

In [51]:
with pl.StringCache():
    agg_yr_cat = agg_yr_sort2.with_columns(
        pl.col('simple_class', 'fuel_type').cast(pl.Categorical)
    )
    inv_yr_cat = inv_yr_sort2.with_columns(
        pl.col('class', 'fuel_type').cast(pl.Categorical)
    )

In [52]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    (
        agg_yr_cat
        .join(
            inv_yr_cat, 
            left_on=['year', 'simple_class', 'fuel_type'], 
            right_on=['year', 'class', 'fuel_type']
        )
    )

timing_test()

501 ms ± 88.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This is much faster.

---
We can compare this to a Pandas sort.


In [53]:
agg_pd = agg_yr_sort2.to_pandas().sort_values(['year', 'simple_class', 'fuel_type'])
inv_pd = inv_yr_sort2.to_pandas().sort_values(['year', 'class','fuel_type'])

In [54]:
agg_pd

,year,simple_class,fuel_type,mean_mpg
0,1984,Car,Diesel vehicle,24.338462
1,1984,Car,Gasoline vehicle,18.221835
2,1984,Other,Diesel vehicle,21.300000
3,1984,Other,Gasoline vehicle,16.907631
4,1984,Truck,Diesel vehicle,20.708333
...,...,...,...,...
440,2025,Other,Gasoline vehicle,20.737430
441,2025,Truck,Diesel vehicle,21.400000
442,2025,Truck,Electric vehicle,78.187500
443,2025,Truck,Flex fuel vehicle,15.000000


In [55]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    (
        agg_pd
        .merge(
            inv_pd, 
            left_on=['year', 'simple_class', 'fuel_type'], 
            right_on=['year', 'class', 'fuel_type']
        )
    )

timing_test()

2.45 s ± 560 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The Pandas sort is much slower!

---

# 2.16 Visualizing the Join
[back to Top](#Top)

* Make a line for each *simple class* vehicle type over the years for Salt Lake City.
* Need a dataframe aggregated by year.
* Use `.pivot()`
* Once we have the year in a column and other columns for each line, make the plot.

In [56]:
# Data prepped for the plot
(
    agg_yr
    .join(
        inv_yr,
        left_on=['year', 'simple_class', 'fuel_type'],
        right_on=['year', 'class', 'fuel_type']
    )
    .filter(city_name='Salt Lake City')
    .pivot(
        index='year',
        on='simple_class',
        values='mean_mpg',
        aggregate_function='mean'
    )
)

join parallel: true
INNER join triggered a rechunk of the left DataFrame: 4 columns are affected
INNER join dataframes finished
dataframe filtered


year,Car,Truck
i16,f64,f64
2000,20.465806,12.558559
2001,19.972288,12.685439
2002,20.834157,12.618696
2003,20.505458,12.45977
2004,19.001403,12.260606
…,…,…
2014,37.992402,16.691919
2015,35.871053,17.464186
2016,34.458447,17.664933


Use .plot.line() for the visualization.

In [57]:
try:
    (
        agg_yr
        .join(
            inv_yr,
            left_on=['year', 'simple_class', 'fuel_type'],
            right_on=['year', 'class', 'fuel_type']
        )
        .filter(city_name='Salt Lake City')
        .pivot(
            index='year',
            on='simple_class',
            values='mean_mpg',
            aggregate_function='mean'
        )
        .plot.line(
            x='year',
            y=['Car', 'Truck'],
        )
    )
except Exception as e:
    print(e)

join parallel: true
INNER join triggered a rechunk of the left DataFrame: 4 columns are affected
INNER join dataframes finished
dataframe filtered


* Encountering `SchemaValidationError` using book-code as-is.

Fix:

* After pivoting, use the melt() function to reshape the data from wide to long format. This creates a column for the vehicle class and a column for the mean MPG values

* In the plot.line() function, we now use:
```
x='year' # for the x-axis
y='mean_mpg' # for the y-axis
color='vehicle_class' # to distinguish between different vehicle classes
```
*  These changes should resolve the SchemaValidationError by providing the correct data structure for the line plot. The resulting plot will show the mean MPG trends over time for different vehicle classes in Salt Lake City 


In [58]:
(
    agg_yr
    .join(
        inv_yr,
        left_on=['year', 'simple_class', 'fuel_type'],
        right_on=['year', 'class', 'fuel_type']
    )
    .filter(pl.col('city_name') == 'Salt Lake City')
    .pivot(
        index='year',
        on='simple_class',
        values='mean_mpg',
        aggregate_function='mean'
    )
    .unpivot(
        index='year', 
        variable_name='vehicle_class', 
        value_name='mean_mpg'
    )
)

join parallel: true
INNER join triggered a rechunk of the left DataFrame: 4 columns are affected
INNER join dataframes finished
dataframe filtered


year,vehicle_class,mean_mpg
i16,str,f64
2000,"""Car""",20.465806
2001,"""Car""",19.972288
2002,"""Car""",20.834157
2003,"""Car""",20.505458
2004,"""Car""",19.001403
…,…,…
2014,"""Truck""",16.691919
2015,"""Truck""",17.464186
2016,"""Truck""",17.664933


In [59]:
(
    agg_yr
    .join(
        inv_yr,
        left_on=['year', 'simple_class', 'fuel_type'],
        right_on=['year', 'class', 'fuel_type']
    )
    .filter(pl.col('city_name') == 'Salt Lake City')
    .pivot(
        index='year',
        on='simple_class',
        values='mean_mpg',
        aggregate_function='mean'
    )
    .unpivot(
        index='year', 
        variable_name='vehicle_class', 
        value_name='mean_mpg'
    )
    .plot.line(
        x='year', 
        y='mean_mpg', 
        color='vehicle_class'
    )
)

join parallel: true
INNER join triggered a rechunk of the left DataFrame: 4 columns are affected
INNER join dataframes finished
dataframe filtered


alt.Chart(...)

---
# 2.17 Adding Rows
[back to Top](#Top)

* To add a row to a dataframe, we can use `.vstack`.
* This takes a dataframe and returns a new dataframe with the rows from the original dataframe, and rows from the dataframe that was passed in.
* This does not modify the original dataframe.

In this example, remove the last 10 rows of the `autos` dataframe, and add the first ten rows to the end of the dataframe.

In [60]:
(
    autos
    .tail(10)
    .vstack(autos.head(10))
)

index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
48192,1993,"""Saab""","""900""",2.1,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",14.8755,18,24,2013-01-01 00:00:00,"""false""",5
48193,1993,"""Saturn""","""SL""",1.9,4,"""Automatic 4-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",11.442693,23,33,2013-01-01 00:00:00,"""true""",4
48194,1993,"""Saturn""","""SL""",1.9,4,"""Automatic 4-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",12.39625,21,30,2013-01-01 00:00:00,"""true""",4
48195,1993,"""Saturn""","""SL""",1.9,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",10.625357,24,33,2013-01-01 00:00:00,"""false""",5
48196,1993,"""Saturn""","""SL""",1.9,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",11.9004,21,32,2013-01-01 00:00:00,"""false""",5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5,1993,"""Subaru""","""Loyale""",1.8,4,"""Automatic 3-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",13.523182,21,24,2013-01-01 00:00:00,"""true""",3
6,1993,"""Subaru""","""Loyale""",1.8,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",11.9004,22,29,2013-01-01 00:00:00,"""false""",5
7,1993,"""Toyota""","""Corolla""",1.6,4,"""Automatic 3-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",12.39625,23,26,2013-01-01 00:00:00,"""true""",3


* You could use `.head()` and `.tail()` to undo a simple `.vstack()` operation if you know how many rows you added.

---
# 2.18 Reshape and Pivoting Data
[back to Top](#Top)

* **One of the most powerful features of Polas is the ability to reshape and pivot data.**
* This allows you to transform data from a **wide format** to a **long format**, and vice versa.
* This is a typical operation in data analysis.
* It is often used to prepare data for plotting or machine learning.

We use `pivot` in this example:
* Filter out the top 5 car manufacturers.
* Pull out values from the *make* column into a new column.
* Make *year* the first column.
* Polars will call this the *index*.
* Take the median of the *city08* column for each *year* and *make* combination.
* This gives us the median city fuel efficiency for each *year* and *make* of car.


In [61]:
# First, determine the top 5 car manufacturers
top_n = (
    autos
    .group_by('make')
    .agg(pl.col('city08').count())
    .sort('city08')
    .tail(5)
)
print(top_n)

shape: (5, 2)
+-----------+--------+
| make      | city08 |
| ---       | ---    |
| cat       | u32    |
+====================+
| BMW       | 2536   |
| Dodge     | 2695   |
| GMC       | 2823   |
| Ford      | 3834   |
| Chevrolet | 4472   |
+-----------+--------+


keys/aggregates are not partitionable: running default HASH AGGREGATION


Next,

* With this expression, filter the dataset for the top-5 car manufacturers.
* Use `.pivot()`
    - Put `year` down the leftmost column.
    - Put `make` across the top.
* **We want city fuel efficiency in the cells**.
* Because there are multiple vehicles for each year and make, we need to aggregate them.
* Use `.median` to reduce multiple values into a single value.
* This table should represent the typical fuel efficiency for each `year` and `make` of car.

In [62]:
print(
    autos
    .filter(pl.col('make').is_in(top_n['make']))
    .pivot(
        index='year', # row-index
        on='make', # columns from make-values
        values='city08', # actual values to extract
        aggregate_function='median' # aggregated operation
    )
)

shape: (42, 6)
+------+-------+------+-----------+------+------+
| year | Dodge | BMW  | Chevrolet | Ford | GMC  |
| ---  | ---   | ---  | ---       | ---  | ---  |
| i16  | f64   | f64  | f64       | f64  | f64  |
+===============================================+
| 1985 | 17.0  | 18.0 | 15.0      | 15.0 | 15.0 |
| 1993 | 14.0  | 15.0 | 15.0      | 15.0 | 14.0 |
| 1994 | 14.5  | 15.5 | 15.0      | 15.0 | 14.0 |
| 1995 | 15.0  | 16.5 | 15.0      | 15.0 | 14.0 |
| 1996 | 13.0  | 18.0 | 15.0      | 15.0 | 14.0 |
| …    | …     | …    | …         | …    | …    |
| 2021 | 15.0  | 21.0 | 16.0      | 20.0 | 15.0 |
| 2022 | 15.0  | 21.0 | 16.0      | 20.0 | 15.0 |
| 2023 | 15.0  | 22.0 | 17.0      | 19.0 | 16.0 |
| 2024 | 16.0  | 23.0 | 17.0      | 19.0 | 16.0 |
| 2025 | 15.5  | 25.0 | 16.0      | 20.0 | 16.0 |
+------+-------+------+-----------+------+------+


dataframe filtered


Notes:

* The `year` column is not sorted.
* This is because Polars runs the group-by operation in parallel, and then combines the results.
* You would get different row orders if you ran the same code multiple times.
* We can address this by sorting the `year` column at the end.

In [63]:
pivoted1 = (
    autos
    .filter(pl.col('make').is_in(top_n['make']))
    .pivot(
        index='year',
        on='make',
        values='city08',
        aggregate_function='median'
    )
    .sort('year')
)

print(pivoted1)

shape: (42, 6)
+------+-------+------+-----------+------+------+
| year | Dodge | BMW  | Chevrolet | Ford | GMC  |
| ---  | ---   | ---  | ---       | ---  | ---  |
| i16  | f64   | f64  | f64       | f64  | f64  |
+===============================================+
| 1984 | 18.0  | 18.0 | 16.0      | 14.5 | 15.0 |
| 1985 | 17.0  | 18.0 | 15.0      | 15.0 | 15.0 |
| 1986 | 14.5  | 15.0 | 15.0      | 16.0 | 15.0 |
| 1987 | 15.0  | 15.0 | 15.0      | 15.0 | 15.0 |
| 1988 | 14.0  | 15.0 | 15.0      | 15.0 | 15.0 |
| …    | …     | …    | …         | …    | …    |
| 2021 | 15.0  | 21.0 | 16.0      | 20.0 | 15.0 |
| 2022 | 15.0  | 21.0 | 16.0      | 20.0 | 15.0 |
| 2023 | 15.0  | 22.0 | 17.0      | 19.0 | 16.0 |
| 2024 | 16.0  | 23.0 | 17.0      | 19.0 | 16.0 |
| 2025 | 15.5  | 25.0 | 16.0      | 20.0 | 16.0 |
+------+-------+------+-----------+------+------+


dataframe filtered


We also have an alternative way of using `.sort()`:

* Tell Polars to preserve the order of the rows by first sorting the `year` column
* Then indicate that the column is sorted with `.set_sorted`.

In [64]:
pivoted2= (
    autos
    .filter(pl.col('make').is_in(top_n['make']))
    .sort('year')
    .set_sorted('year')
    .pivot(
        index='year',
        on='make',
        values='city08',
        aggregate_function='median'
    )
)

print(pivoted2)

shape: (42, 6)
+------+-----------+------+------+-------+------+
| year | Chevrolet | Ford | BMW  | Dodge | GMC  |
| ---  | ---       | ---  | ---  | ---   | ---  |
| i16  | f64       | f64  | f64  | f64   | f64  |
+===============================================+
| 1984 | 16.0      | 14.5 | 18.0 | 18.0  | 15.0 |
| 1985 | 15.0      | 15.0 | 18.0 | 17.0  | 15.0 |
| 1986 | 15.0      | 16.0 | 15.0 | 14.5  | 15.0 |
| 1987 | 15.0      | 15.0 | 15.0 | 15.0  | 15.0 |
| 1988 | 15.0      | 15.0 | 15.0 | 14.0  | 15.0 |
| …    | …         | …    | …    | …     | …    |
| 2021 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2022 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2023 | 17.0      | 19.0 | 22.0 | 15.0  | 16.0 |
| 2024 | 17.0      | 19.0 | 23.0 | 16.0  | 16.0 |
| 2025 | 16.0      | 20.0 | 25.0 | 15.5  | 16.0 |
+------+-----------+------+------+-------+------+


dataframe filtered


---
There is also an additional option:

* Indicate in `.pivot()` that the result should be in the order of the index column.
* For this, use `maintain_order=True` in `.pivot()`

In [65]:
pivoted3 = (
    autos
    .filter(pl.col('make').is_in(top_n['make']))
    .sort('year')
    .pivot(
        index='year',
        on='make',
        values='city08',
        aggregate_function='median',
        maintain_order=True
    )
)
print(pivoted3)

shape: (42, 6)
+------+-----------+------+------+-------+------+
| year | Chevrolet | Ford | BMW  | Dodge | GMC  |
| ---  | ---       | ---  | ---  | ---   | ---  |
| i16  | f64       | f64  | f64  | f64   | f64  |
+===============================================+
| 1984 | 16.0      | 14.5 | 18.0 | 18.0  | 15.0 |
| 1985 | 15.0      | 15.0 | 18.0 | 17.0  | 15.0 |
| 1986 | 15.0      | 16.0 | 15.0 | 14.5  | 15.0 |
| 1987 | 15.0      | 15.0 | 15.0 | 15.0  | 15.0 |
| 1988 | 15.0      | 15.0 | 15.0 | 14.0  | 15.0 |
| …    | …         | …    | …    | …     | …    |
| 2021 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2022 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2023 | 17.0      | 19.0 | 22.0 | 15.0  | 16.0 |
| 2024 | 17.0      | 19.0 | 23.0 | 16.0  | 16.0 |
| 2025 | 16.0      | 20.0 | 25.0 | 15.5  | 16.0 |
+------+-----------+------+------+-------+------+


dataframe filtered


---
# 2.19 Melted Data
[back to Top](#Top)

* Melted data is the opposite of pivoting data.
* It transform data from a wide format to a long format ("tidy data")
* `.melt()` has been deprecated; use `.unpivot()` instead.

Here we want to unpivot the data from the previous example.

* Pass at least 2 parameters to `.unpivot()`:
    1. `index` parameter is a list of columns we want to keep.
        - `index='year'`
    2. `value_vars` parameter lists columns we want to melt.
        - `value_vars=['Chevrolet', 'Ford', 'BMW', 'Dodge', 'GMC']`

In [66]:
# Original pivoted data (before melting)
print(pivoted3)

shape: (42, 6)
+------+-----------+------+------+-------+------+
| year | Chevrolet | Ford | BMW  | Dodge | GMC  |
| ---  | ---       | ---  | ---  | ---   | ---  |
| i16  | f64       | f64  | f64  | f64   | f64  |
+===============================================+
| 1984 | 16.0      | 14.5 | 18.0 | 18.0  | 15.0 |
| 1985 | 15.0      | 15.0 | 18.0 | 17.0  | 15.0 |
| 1986 | 15.0      | 16.0 | 15.0 | 14.5  | 15.0 |
| 1987 | 15.0      | 15.0 | 15.0 | 15.0  | 15.0 |
| 1988 | 15.0      | 15.0 | 15.0 | 14.0  | 15.0 |
| …    | …         | …    | …    | …     | …    |
| 2021 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2022 | 16.0      | 20.0 | 21.0 | 15.0  | 15.0 |
| 2023 | 17.0      | 19.0 | 22.0 | 15.0  | 16.0 |
| 2024 | 17.0      | 19.0 | 23.0 | 16.0  | 16.0 |
| 2025 | 16.0      | 20.0 | 25.0 | 15.5  | 16.0 |
+------+-----------+------+------+-------+------+


In [67]:
pivoted3.columns

['year', 'Chevrolet', 'Ford', 'BMW', 'Dodge', 'GMC']

In [68]:
# using deprecated 'melt' with default arguments
print(
    pivoted3.melt()
)

shape: (252, 2)
+----------+--------+
| variable | value  |
| ---      | ---    |
| str      | f64    |
+===================+
| year     | 1984.0 |
| year     | 1985.0 |
| year     | 1986.0 |
| year     | 1987.0 |
| year     | 1988.0 |
| …        | …      |
| GMC      | 15.0   |
| GMC      | 15.0   |
| GMC      | 16.0   |
| GMC      | 16.0   |
| GMC      | 16.0   |
+----------+--------+


/var/folders/sb/p6dz60l562q8pzfgzf74gsxm0000gn/T/ipykernel_1929/1126281606.py:3: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  pivoted3.melt()


In [69]:
# With new unpivot with default arguments
print(
    pivoted3.unpivot()
)

# DeprecationWarning: `DataFrame.melt` is deprecated. 
# Use `unpivot` instead, with `index` instead of `id_vars` and 
# `on` instead of `value_vars` melt()

shape: (252, 2)
+----------+--------+
| variable | value  |
| ---      | ---    |
| str      | f64    |
+===================+
| year     | 1984.0 |
| year     | 1985.0 |
| year     | 1986.0 |
| year     | 1987.0 |
| year     | 1988.0 |
| …        | …      |
| GMC      | 15.0   |
| GMC      | 15.0   |
| GMC      | 16.0   |
| GMC      | 16.0   |
| GMC      | 16.0   |
+----------+--------+


In [70]:
# Using id_vars argument
print(
    pivoted3.unpivot(index='year')
)

shape: (210, 3)
+------+-----------+-------+
| year | variable  | value |
| ---  | ---       | ---   |
| i16  | str       | f64   |
+==========================+
| 1984 | Chevrolet | 16.0  |
| 1985 | Chevrolet | 15.0  |
| 1986 | Chevrolet | 15.0  |
| 1987 | Chevrolet | 15.0  |
| 1988 | Chevrolet | 15.0  |
| …    | …         | …     |
| 2021 | GMC       | 15.0  |
| 2022 | GMC       | 15.0  |
| 2023 | GMC       | 16.0  |
| 2024 | GMC       | 16.0  |
| 2025 | GMC       | 16.0  |
+------+-----------+-------+


In [71]:
# Using id_vars and value_vars argument
print(
    pivoted3.unpivot(
        index='year',
        on=['Chevrolet', 'Ford', 'BMW', 'Dodge', 'GMC']
    )
)

shape: (210, 3)
+------+-----------+-------+
| year | variable  | value |
| ---  | ---       | ---   |
| i16  | str       | f64   |
+==========================+
| 1984 | Chevrolet | 16.0  |
| 1985 | Chevrolet | 15.0  |
| 1986 | Chevrolet | 15.0  |
| 1987 | Chevrolet | 15.0  |
| 1988 | Chevrolet | 15.0  |
| …    | …         | …     |
| 2021 | GMC       | 15.0  |
| 2022 | GMC       | 15.0  |
| 2023 | GMC       | 16.0  |
| 2024 | GMC       | 16.0  |
| 2025 | GMC       | 16.0  |
+------+-----------+-------+


**Notes**:

* The dataframe has default names of `variable` and `value` for the columns.
* We can change them with the `variable_name` and `value_name` parameters.
* The `variable_name` is the name of the column that contains the melted column names. Here, it is the `make` column.
* The `value_name` parameter is the name of the column that contains the melted values. Here, it is the `city08` column. 

In [72]:
print(
    pivoted3
    .unpivot(
        index='year',
        on=['Chevrolet', 'Ford', 'BMW', 'Dodge', 'GMC'],
        variable_name='make',
        value_name='median_city_mpg',
    )
)

shape: (210, 3)
+------+-----------+-----------------+
| year | make      | median_city_mpg |
| ---  | ---       | ---             |
| i16  | str       | f64             |
+====================================+
| 1984 | Chevrolet | 16.0            |
| 1985 | Chevrolet | 15.0            |
| 1986 | Chevrolet | 15.0            |
| 1987 | Chevrolet | 15.0            |
| 1988 | Chevrolet | 15.0            |
| …    | …         | …               |
| 2021 | GMC       | 15.0            |
| 2022 | GMC       | 15.0            |
| 2023 | GMC       | 16.0            |
| 2024 | GMC       | 16.0            |
| 2025 | GMC       | 16.0            |
+------+-----------+-----------------+


---
# 2.20 Finding Duplicates
[back to Top](#Top)

* `.join()` allows us to validate the join type.
* This can help catch errors where we have a one-to-many or many-to-many join when we expect a 1-to-1 join.
* However, sometimes we have data from an external source and want to validate the data is correct and there are no duplicates.
* We can use `.is_duplicated()` on both the dataframe and column expression.
* We usually want to combine an expression returning boolean values with `.filter()` to find duplicate rows.

*Example*
* Here, we check for duplicate rows in the autos dataset.

In [73]:
(
    autos
    # selects works in a column-basis
    .select(
        pl.col('highway08').is_duplicated()
    )
)

highway08
bool
true
true
true
true
true
…
true
true
true


* We can view the rows with duplicate values using `.filter()`.
* Pass in the boolean mask we created with `.is_duplicated()`.

In [74]:
(
    autos
    # filter works vertically on a row basis (eg .loc, .iloc, query in Pandas)
    .filter(pl.col('highway08').is_duplicated())
)

dataframe filtered


index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
0,1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",14.167143,19,25,2013-01-01 00:00:00,"""false""",5
1,1985,"""Ferrari""","""Testarossa""",4.9,12,"""Manual 5-spd""","""Rear-Wheel Drive""","""Two Seaters""","""Regular""",27.046364,9,14,2013-01-01 00:00:00,"""false""",5
2,1985,"""Dodge""","""Charger""",2.2,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Subcompact Cars""","""Regular""",11.018888,23,33,2013-01-01 00:00:00,"""false""",5
3,1985,"""Dodge""","""B150/B250 Wagon 2WD""",5.2,8,"""Automatic 3-spd""","""Rear-Wheel Drive""","""Vans""","""Regular""",27.046364,10,12,2013-01-01 00:00:00,"""true""",3
4,1993,"""Subaru""","""Legacy AWD Turbo""",2.2,4,"""Manual 5-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Premium""",15.658422,17,23,2013-01-01 00:00:00,"""false""",5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
48197,1993,"""Subaru""","""Legacy""",2.2,4,"""Automatic 4-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",13.523182,19,26,2013-01-01 00:00:00,"""true""",4
48198,1993,"""Subaru""","""Legacy""",2.2,4,"""Manual 5-spd""","""Front-Wheel Drive""","""Compact Cars""","""Regular""",12.935218,20,28,2013-01-01 00:00:00,"""false""",5
48199,1993,"""Subaru""","""Legacy AWD""",2.2,4,"""Automatic 4-spd""","""4-Wheel or All-Wheel Drive""","""Compact Cars""","""Regular""",14.167143,18,24,2013-01-01 00:00:00,"""true""",4


* This isn't particulary interesting, since many cars have the same highway fuel efficiency.
* A more realistic example is checking for duplicate year-model combinations.
* We cannot include both columns in the `pl.col` expression like `pl.col('year', 'model').is_duplicated()` as this will throw an error.
* If we want to determine where a combination of columns is duplicated as a group, we need to combine them.
* We can do this with a *struct* expression.
* In Polars, a *struct* is a single column that contains a sequence of values (the collection of columns that are combined).
* We will check for duplicates of *year* *model* combinations.
* First, combine the two columns into a single column using `pl.struct()`.

In [75]:
# If we use .to_struct without any arguments, 
# all columns are pulled into a row-wise struct
all_structs1 = autos.to_struct()

print(type(all_structs1))
HR()
print(all_structs1.head(2))

<class 'polars.series.series.Series'>
----------------------------------------
shape: (2,)
Series: '' [struct[16]]
[
	{0,1985,"Alfa Romeo","Spider Veloce 2000",2.0,4,"Manual 5-spd","Rear-Wheel Drive","Two Seaters","Regular",14.167143,19,25,2013-01-01 00:00:00,"false",5}
	{1,1985,"Ferrari","Testarossa",4.9,12,"Manual 5-spd","Rear-Wheel Drive","Two Seaters","Regular",27.046364,9,14,2013-01-01 00:00:00,"false",5}
]


In [76]:
# Create structs but in DataFrame format
all_structs2 = autos.select(pl.struct(pl.all()))

print(type(all_structs2))
HR()
print(all_structs2.head(2))

<class 'polars.dataframe.frame.DataFrame'>
----------------------------------------
shape: (2, 1)
+---------------------------------+
| index                           |
| ---                             |
| struct[16]                      |
+=================================+
| {0,1985,"Alfa Romeo","Spider V… |
| {1,1985,"Ferrari","Testarossa"… |
+---------------------------------+


In [77]:
try:
    all_structs1 == all_structs2
except Exception as e:
    print(e)

cannot convert Python type 'DataFrame' to Struct({'index': UInt32, 'year': Int16, 'make': Categorical(ordering='physical'), 'model': Categorical(ordering='physical'), 'displ': Float32, 'cylinders': UInt8, 'trany': String, 'drive': Categorical(ordering='physical'), 'VClass': Categorical(ordering='physical'), 'fuelType': Categorical(ordering='physical'), 'barrels08': Float32, 'city08': UInt8, 'highway08': UInt8, 'createdOn': Datetime(time_unit='us', time_zone=None), 'is_automatic': String, 'num_gears': UInt8})


In [78]:
pl.struct(pl.col('year', 'model'))

<Expr ['cols(["year", "model"]).as_str…'] at 0x128113E90>

In [79]:
pl.struct(pl.col('year'), pl.col('model'))

<Expr ['col("year").as_struct([col("mo…'] at 0x13E3D72D0>

In [80]:
struct1 = (
    autos
    .select(
        pl.struct(pl.col('year'), pl.col('model'))
    )
)
struct1

year
struct[2]
"{1985,""Spider Veloce 2000""}"
"{1985,""Testarossa""}"
"{1985,""Charger""}"
"{1985,""B150/B250 Wagon 2WD""}"
"{1993,""Legacy AWD Turbo""}"
…
"{1993,""Legacy""}"
"{1993,""Legacy""}"
"{1993,""Legacy AWD""}"


In [81]:
struct2 = (
    autos
    .select(
        pl.struct(pl.col(['year', 'model']))
    )
)
struct2

year
struct[2]
"{1985,""Spider Veloce 2000""}"
"{1985,""Testarossa""}"
"{1985,""Charger""}"
"{1985,""B150/B250 Wagon 2WD""}"
"{1993,""Legacy AWD Turbo""}"
…
"{1993,""Legacy""}"
"{1993,""Legacy""}"
"{1993,""Legacy AWD""}"


In [82]:
struct1.equals(struct2)

True

---
Now we can use `.is_duplicated()` to find the duplicate values.

In [83]:
(
    autos
    .select(pl.struct(pl.col(['year', 'model'])).is_duplicated())
)

year
bool
false
false
true
true
true
…
true
true
true


---
We can then find the duplicate rows by replacing `.select` with `.filter`

GB: Conceptually, this allows us to find popular combinations, eg Jones Solution in size 164cm, etc

In [84]:
(
    autos
    .filter(pl.struct(pl.col(['year', 'model'])).is_duplicated())
    .sort('year', 'model')
)

dataframe filtered


index,year,make,model,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,createdOn,is_automatic,num_gears
u32,i16,cat,cat,f32,u8,str,cat,cat,cat,f32,u8,u8,datetime[μs],str,u8
18214,1984,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""",null,"""Two Seaters""","""Regular""",14.167143,18,25,2013-01-01 00:00:00,"""false""",5
19458,1984,"""Alfa Romeo""","""Spider Veloce 2000""",2.0,4,"""Manual 5-spd""",null,"""Two Seaters""","""Regular""",14.167143,18,25,2013-01-01 00:00:00,"""false""",5
19555,1984,"""Dodge""","""Charger""",1.6,4,"""Manual 4-spd""",null,"""Subcompact Cars""","""Regular""",10.258965,25,35,2013-01-01 00:00:00,"""false""",4
18318,1984,"""Dodge""","""Charger""",1.6,4,"""Manual 4-spd""",null,"""Subcompact Cars""","""Regular""",9.917,26,35,2013-01-01 00:00:00,"""false""",4
18319,1984,"""Dodge""","""Charger""",2.2,4,"""Automatic 3-spd""",null,"""Subcompact Cars""","""Regular""",12.935218,21,26,2013-01-01 00:00:00,"""true""",3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41457,2025,"""Lexus""","""UX 300h AWD""",2.0,4,"""Automatic (AV-S6)""","""All-Wheel Drive""","""Compact Cars""","""Regular""",7.083571,44,40,2024-03-19 00:00:00,"""true""",6
41925,2025,"""Mazda""","""CX-70 4WD""",3.3,6,"""Automatic (S8)""","""4-Wheel Drive""","""Standard Sport Utility Vehicle…","""Premium""",11.9004,23,28,2024-08-07 00:00:00,"""true""",8
42102,2025,"""Mazda""","""CX-70 4WD""",3.3,6,"""Automatic (S8)""","""4-Wheel Drive""","""Standard Sport Utility Vehicle…","""Regular""",11.9004,24,28,2024-08-25 00:00:00,"""true""",8


We can see there are multiple *year* *model* combinations that are duplicated.

---
# 2.21 Finding Missing Values
[back to Top](#Top)

* A big difference between Pandas and Polars is Polars distinguishes between NULL and NaN values.
    - A NULL value is a value that is missing.
    - A NaN value is a value that is not a number and may result from a mathematical operation.
* Non-numerical data types cannot be NaN types.
* For ex, a string cannot be a NaN type.
* Polars has both `.is_null()` and `.is_nan()` that are found on a column expression.
* When reading a CSV file, it is possible to get both NULL and NaN values in a single column.

Here, we use synthetic data to demonstrate the difference between NULL and NaN values.

In [85]:
missing_df = pl.DataFrame({'val': [-1.1, 0, 2.3, None, 5.7, 7]})
print(missing_df)

shape: (6, 1)
+------+
| val  |
| ---  |
| f64  |
+======+
| -1.1 |
| 0.0  |
| 2.3  |
| null |
| 5.7  |
| 7.0  |
+------+


We can see a null value here. See what happens when we try to divide the `val` column by itself.

In [86]:
(
    missing_df
    .with_columns(
        val2=pl.col('val') / pl.col('val')
    )
)

val,val2
f64,f64
-1.1,1.0
0.0,NaN
2.3,1.0
null,null
5.7,1.0
7.0,1.0


When 0 is divided by 0, the result is NaN. 

NaN in Polars means it is not a number (eg the result of an invalid math operation).

Null simply means a missing value.

There are several expressions that deal with missing values and NaNs: `.is_finite`, `.is_nan`, `.is_null`.

Here, we use predicate methods that deal with missing values and NaNs to see how they work on a column that has both NaN and null values.

In [87]:
(
    missing_df
    .with_columns(val2=pl.col('val')/pl.col('val'))
    .with_columns(
        is_null2=pl.col('val2').is_null(),
        is_nan2=pl.col('val2').is_nan(),
        is_finite2=pl.col('val2').is_finite(),
        interpolate=pl.col('val2').interpolate()
    )
    
)

val,val2,is_null2,is_nan2,is_finite2,interpolate
f64,f64,bool,bool,bool,f64
-1.1,1.0,false,false,true,1.0
0.0,NaN,false,true,false,NaN
2.3,1.0,false,false,true,1.0
null,null,true,null,null,1.0
5.7,1.0,false,false,true,1.0
7.0,1.0,false,false,true,1.0


We can use `.fill_null` to replace the NULL values in the *val* column with 0.

In [88]:
(
    missing_df
    .with_columns(
        forward=pl.col('val').fill_null(strategy='forward'),
        backward=pl.col('val').fill_null(strategy='backward'),
        min=pl.col('val').fill_null(strategy='min'),
        max=pl.col('val').fill_null(strategy='max')
    )
)

val,forward,backward,min,max
f64,f64,f64,f64,f64
-1.1,-1.1,-1.1,-1.1,-1.1
0.0,0.0,0.0,0.0,0.0
2.3,2.3,2.3,2.3,2.3
null,2.3,5.7,-1.1,7.0
5.7,5.7,5.7,5.7,5.7
7.0,7.0,7.0,7.0,7.0


This is an example of using `.fill_null()` and `.interpolate`.

In [89]:
(
    missing_df
    .with_columns(
        mean=pl.col('val').fill_null(pl.col('val').mean()),
        interpolate=pl.col('val').interpolate(),
        nearest=pl.col('val').interpolate('nearest')
    )
)

val,mean,interpolate,nearest
f64,f64,f64,f64
-1.1,-1.1,-1.1,-1.1
0.0,0.0,0.0,0.0
2.3,2.3,2.3,2.3
null,2.78,4.0,5.7
5.7,5.7,5.7,5.7
7.0,7.0,7.0,7.0


Experiment with saving to a CSV and reading it again.

In [90]:
(
    missing_df
    .with_columns(
        val2=pl.col('val')/pl.col('val')
    )
    .write_csv('./data/missing.csv')
)

In [91]:
%pfile ./data/missing.csv

Object `./data/missing.csv` not found.


val,val2
-1.1,1.0
0.0,NaN
2.3,1.0
,
5.7,1.0
7.0,1.0


In [92]:
pl.read_csv('./data/missing.csv')

file < 128 rows, no statistics determined
no. of chunks: 1 processed by: 1 threads.


val,val2
f64,f64
-1.1,1.0
0.0,NaN
2.3,1.0
null,null
5.7,1.0
7.0,1.0


* Polars is able to parse this correctly.
* It can distinguish between NaN and null values.
* If we read this with Pandas, it will convert both of the values to Nan.

As a test, we can convert the dataframe to Pandas with `.to_pandas()`

In [93]:
# Pandas
(
    missing_df
    .with_columns(val2=pl.col('val')/pl.col('val'))
    .to_pandas()
)

,val,val2
0,-1.1,1.0
1,0.0,NaN
2,2.3,1.0
3,NaN,NaN
4,5.7,1.0
5,7.0,1.0


In [94]:
# Pandas
pd.read_csv('./data/missing.csv')

,val,val2
0,-1.1,1.0
1,0.0,NaN
2,2.3,1.0
3,NaN,NaN
4,5.7,1.0
5,7.0,1.0


---
# 2.22 Third-Party Libraries and Missing Values
[back to Top](#Top)

* Explore how libraries like XGBoost and NumPy deal with Polars data that has NaN and null values.
* We will use the XGBoost library to make a regression model.
* We train it on the synthetic data that has both NaN and null values.
* XGBoost supports missing values in Pandas data.

In [95]:
X = (
    missing_df
    .with_columns(val2=pl.col('val')/pl.col('val'))
)
X

val,val2
f64,f64
-1.1,1.0
0.0,NaN
2.3,1.0
null,null
5.7,1.0
7.0,1.0


Labels for the regression model.

In [96]:
y = pl.Series(range(6))
y

""
i64
0
1
2
3
4
5


Train the model.

In [97]:
import xgboost as xgb

reg = xgb.XGBRegressor()
reg.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

From the above, the model trained with no problems.

Make a prediction using the training data.

In [98]:
reg.predict(X)

array([1.1198493e-03, 1.0000179e+00, 1.9999869e+00, 3.0000448e+00,
       3.9999456e+00, 4.9988852e+00], dtype=float32)

XGBoost works fine with Polars numeric values. Note that it returns a NumPy array.

---

Next, we will try the NumPy library.
Take the sine of the X variable.

In [99]:
import numpy as np

np.sin(X)

array([[-0.89120736,  0.84147098],
       [ 0.        ,         nan],
       [ 0.74570521,  0.84147098],
       [        nan,         nan],
       [-0.55068554,  0.84147098],
       [ 0.6569866 ,  0.84147098]])

**Notes**

* NumPy treats the NaN and null as nan. 
* Polars represents missing values using a bitmask to indicate that values are missing. 
* NumPy doesn't have this concept.

---
# 2.23 Additional Column Selectors and Missing Values
[back to Top](#Top)

* Examine the numeric rows from the *autos* data where any value is null.
* Use the selectors module to select the numeric columns.
* It has a numeric function that returns all numeric columns in a Dataframe.

--- 
## Notes: polars.selectors

Polars.selectors is a module in the Polars Python library that provides a convenient and intuitive way to select columns from DataFrame or LazyFrame objects based on various criteria such as column names, data types, or other properties. It unifies and extends the functionality available through the `col()` expression, allowing for more expressive and composable column selection.

**Key Features of Polars.selectors**

1. **Column Selection by Name**: You can select columns by their names using `cs.by_name()`. For example, `df.select(cs.by_name("col1", "col2"))` selects columns named "col1" and "col2".

2. **Column Selection by Data Type**: You can select columns based on their data types using `cs.by_dtype()`. For example, `df.select(cs.by_dtype(pl.Int64))` selects all integer columns, and `df.select(cs.numeric())` selects all numeric columns (integers and floats).

3. **Positional Selection**: You can select the first or last column using `cs.first()` and `cs.last()`, respectively.

4. **Set Operations**: Selectors support set operations like union (`|`), intersection (`&`), difference (`-`), and complement (`~`). For example, `df.select(cs.numeric() | cs.string())` selects all numeric and string columns.

5. **Composability**: Selectors can be combined with expressions and applied to the selected columns. For example, `df.select(cs.numeric().sum())` sums all numeric columns.

6. **Expansion**: You can expand a selector to column names with respect to a specific DataFrame, LazyFrame, or schema using `cs.expand_selector()`.

## Notes: Rationale for Separating Selectors and Expressions

While selectors are a type of expression and use expressions internally, they are kept separate from the `col()` expressions for several reasons:

1. **Clarity**: Separating selectors and expressions makes the API more intuitive and easier to understand, especially for new users.

2. **Distinct Behavior**: Selectors have distinct behaviors and properties that differ from standard expressions, such as set operations and column expansion.

3. **Composability**: Keeping selectors separate allows for better composability and easier extension of the selector functionality.

4. **Ambiguity Avoidance**: Merging selectors and expressions could lead to ambiguity and unexpected behavior when dealing with operations that involve both types of objects.

---
## Notes: Difference between Polars expressions and selectors

Selectors and expressions in the Polars library are related but distinct concepts that serve different purposes:

**Expressions**

Expressions in Polars represent computations or transformations on columns or data. They are created using the `pl.col()` function or by combining other expressions with operators or methods. Expressions can perform various operations like arithmetic, comparisons, string manipulations, etc.

Examples:
- `pl.col("col1") + pl.col("col2")` (addition of two columns)
- `pl.col("col1").str.contains("pattern")` (string operation on a column)

**Selectors**

Selectors, on the other hand, are used to select or filter columns from a DataFrame or LazyFrame based on certain criteria like column names, data types, or other properties. They provide a more intuitive and composable way to select columns compared to using expressions directly.

Key differences between selectors and expressions:

1. **Purpose**: Expressions represent computations, while selectors are used for column selection/filtering.

2. **Creation**: Expressions are created using `pl.col()` or combining other expressions, while selectors are imported from `polars.selectors` (typically as `cs`).

3. **Set Operations**: Selectors support set operations like union (`|`), intersection (`&`), difference (`-`), and complement (`~`) for combining or manipulating selections, which is not directly possible with expressions.

4. **Composability**: Selectors can be easily composed and combined with expressions to perform operations on the selected columns. For example, `df.select(cs.numeric().sum())` sums all numeric columns.

5. **Expansion**: Selectors can be expanded to column names with respect to a specific DataFrame, LazyFrame, or schema using `cs.expand_selector()`.

6. **Separation of Concerns**: Keeping selectors separate from expressions makes the API more intuitive, avoids ambiguity, and allows for better composability and extensibility of the selector functionality.

While selectors are based on expressions internally, they provide a higher-level abstraction for column selection and manipulation, making the API more user-friendly and expressive. The separation between selectors and expressions is intentional in the Polars library to improve clarity, composability, and extensibility of the API.

In [100]:
import polars.selectors as cs

In [101]:
[x for x in dir(cs) if not x.startswith('_')]

['Any',
 'Binary',
 'Boolean',
 'Categorical',
 'Collection',
 'Date',
 'Datetime',
 'Decimal',
 'Duration',
 'Expr',
 'F',
 'FLOAT_DTYPES',
 'INTEGER_DTYPES',
 'Literal',
 'Mapping',
 'NUMERIC_DTYPES',
 'NoReturn',
 'Object',
 'SIGNED_INTEGER_DTYPES',
 'Sequence',
 'String',
 'TEMPORAL_DTYPES',
 'TYPE_CHECKING',
 'Time',
 'UNSIGNED_INTEGER_DTYPES',
 'all',
 'alpha',
 'alphanumeric',
 'annotations',
 'binary',
 'boolean',
 'by_dtype',
 'by_index',
 'by_name',
 'categorical',
 'contains',
 'date',
 'datetime',
 'decimal',
 'digit',
 'duration',
 'ends_with',
 'exclude',
 'expand_selector',
 'first',
 'float',
 'integer',
 'is_column',
 'is_polars_dtype',
 'is_selector',
 'last',
 'matches',
 'numeric',
 'object',
 'or_',
 'overload',
 're_escape',
 'reduce',
 'signed_integer',
 'starts_with',
 'string',
 'temporal',
 'time',
 'timezone',
 'unsigned_integer']

In [102]:
(
    autos
    .select(cs.numeric().is_null().sum())
)

index,year,displ,cylinders,barrels08,city08,highway08,num_gears
u32,u32,u32,u32,u32,u32,u32,u32
0,0,963,965,0,0,0,0


The `selectors` module has many convenience functions for selecitn columns. 

For ex, if you wanted to select all integer columns, you can use the `integer` function.

Here are other polar.selectors available:

    all
    binary
    boolean
    categorical
    date
    datetime
    float
    integer
    numeric
    object
    string
    temporal
    time
    timezone


* You can also perform set operations on the selectors.
* You can use + and | to perform a union operation.
* You can use - to perform a set difference operation.
* You can use & to perform an intersection operation.
* You can use ^ to perform a symmetric difference operation.

In this example to select all columns that are not dates or booleans:
* Start with all columns
* Remove any date columns
* Remove any boolean columns using set difference


In [103]:
no_dates_or_booleans = (
    cs.all() - cs.date() - cs.boolean()
)
no_dates_or_booleans

((cs.all() - cs.date()) - cs.boolean())

You can debug the selectors with `expand_selector()`. This will show you which columns are selected by the selector.

In [104]:
cs.expand_selector(autos, no_dates_or_booleans)

('index',
 'year',
 'make',
 'model',
 'displ',
 'cylinders',
 'trany',
 'drive',
 'VClass',
 'fuelType',
 'barrels08',
 'city08',
 'highway08',
 'createdOn',
 'is_automatic',
 'num_gears')

We can fill in the missing values of *displ* and *cylinders* with the mean of each year's *make* column. 

First, create an expression to perform that calculation.

---

**polars.Expr.over**

* https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.over.html

Compute expressions over the given groups.

This expression is similar to performing a group by aggregation and joining the result back into the original DataFrame.

The outcome is similar to how window functions work in PostgreSQL.

In [105]:
(
    autos
    .select(pl.col('displ').mean().over(['year', 'make']))
)

displ
f32
2.25
3.566667
3.33211
3.33211
1.951515
…
1.951515
1.951515
1.951515


Now, debug by looking at the rows where the *displ* is null.

In [106]:
# Fill in the missing values of displ and cylinders with the mean
# of each year's make column.
(
    autos
    .with_columns(
        # Create mean for each year and make combination
        mean_displ = pl.col('displ').mean().over(['year', 'make'])
    )
    
    # Use .filter to select rows where displ column is null
    .filter(pl.col('displ').is_null())
    
    # Use .select to limit the columns to the make, year, displ and mean_displ columns
    .select(['year', 'make', 'displ', 'mean_displ'])
)

dataframe filtered


year,make,displ,mean_displ
i16,cat,f32,f32
2000,"""Nissan""",null,2.836
2000,"""Toyota""",null,2.644444
2001,"""Toyota""",null,2.736538
2001,"""Ford""",null,3.837362
2001,"""Ford""",null,3.837362
…,…,…,…
2024,"""Mercedes-Benz""",null,2.90566
2024,"""Tesla""",null,null
2024,"""Tesla""",null,null


In [107]:
# Also check for null values in mean_displ
(
    autos
    .with_columns(
        # Create mean for each year and make combination
        mean_displ = pl.col('displ').mean().over(['year', 'make'])
    )
    # Use .filter to select rows where displ column is null
    .filter(pl.col('mean_displ').is_null())
    # Use .select to limit the columns to the make, year, displ and mean_displ columns
    .select(['year', 'make', 'displ', 'mean_displ'])
    .select(pl.col('make')).to_series().value_counts(sort=True)
)

dataframe filtered


make,count
cat,u32
"""Tesla""",148
"""Rivian""",91
"""Lucid""",33
"""Polestar""",22
"""BYD""",7
…,…
"""Fiat""",2
"""Bugatti Rimac""",2
"""Azure Dynamics""",1


**Notes**

* We can see null values for `mean_displ` for Tesla, Revian, etc.
* This is because none of these cars have internal combustion engines.
* We can address this by calling `.fill_null(0)` on the *mean_displ* column.

In [108]:
mean_displ = (
    pl.col('displ')
    .mean().over(['year', 'make'])
    .fill_null(0)
    .alias('mean_displ')
)

mean_displ

<Expr ['col("displ").mean().over([col(…'] at 0x201F7E850>

In [109]:
(
    autos
    .with_columns(mean_displ)
    .filter(pl.col('displ').is_null()) # include rows that are null in this column
    .select(['year', 'make', 'displ', 'mean_displ'])
)

dataframe filtered


year,make,displ,mean_displ
i16,cat,f32,f32
2000,"""Nissan""",null,2.836
2000,"""Toyota""",null,2.644444
2001,"""Toyota""",null,2.736538
2001,"""Ford""",null,3.837362
2001,"""Ford""",null,3.837362
…,…,…,…
2024,"""Mercedes-Benz""",null,2.90566
2024,"""Tesla""",null,0.0
2024,"""Tesla""",null,0.0


---
Finally, create a new dataframe without missing *displ* values.

For this we fill in the missing values with the *mean_displ* column.

We need to pass the expression we stored in the `mean_displ` variable to `.fill_null()`.

In [110]:
(
    autos
    .with_columns(pl.col('displ').fill_null(mean_displ))
    .select(['year', 'make', 'model', 'displ'])
)

year,make,model,displ
i16,cat,cat,f32
1985,"""Alfa Romeo""","""Spider Veloce 2000""",2.0
1985,"""Ferrari""","""Testarossa""",4.9
1985,"""Dodge""","""Charger""",2.2
1985,"""Dodge""","""B150/B250 Wagon 2WD""",5.2
1993,"""Subaru""","""Legacy AWD Turbo""",2.2
…,…,…,…
1993,"""Subaru""","""Legacy""",2.2
1993,"""Subaru""","""Legacy""",2.2
1993,"""Subaru""","""Legacy AWD""",2.2


**Notes**

This is example of how to debug and work out expressions needed to manipulate data.

* Start with `.select`
* Convert it to a `.with_columns` to add new columns
* Use `.filter` to select values to explore with boolean expressions.

---
# 2.24 Map and Apply
[back to Top](#Top)

If you need to to integrate Polars with a library that doesn't understand Polars expressions, you can use `.map_batches()` to pass a series to a custom function.

Here we make a debug function that prints out the type of object passed to it.
This helps to understand what happens when passing a Polars series to a custom function.


**Notes**

`.map_batches` is a method in Polars that allows you to apply a custom Python function to a whole Series or a sequence of Series. It is useful for transforming the values represented by an expression using a third-party library or performing complex operations that cannot be expressed directly in Polars' expression syntax.

The key points about `.map_batches` are:

1. It takes a function as an argument that operates on one or more Series as input and returns a single Series as output. The function can be a lambda or a regular Python function.

2. It can be used in both a regular DataFrame context as well as in a grouped context (after `groupby`).

3. In a grouped context, you can use the `agg_list` parameter to control whether the function is applied to each group individually (when `agg_list=False`) or to the aggregated list of values across all groups (when `agg_list=True`). Using `agg_list=True` can improve performance for certain operations.

4. The `return_dtype` parameter specifies the data type of the output Series. If not provided, it will be inferred from the first non-null value returned by the function.

5. There is some confusion around whether `.map_batches` works with window functions like `over()`. The documentation doesn't explicitly mention this, and there are open issues discussing this behavior.

This will produce a new DataFrame with three columns: "a", "b", and "a+b+c", where the last column is the result of applying `test_func` to the corresponding "a" and "b" values plus 1.

In [111]:
def debug(thing):
    print(type(thing))
    return thing

(
    autos
    .select(
        pl.col('make').map_batches(debug)
    )
)

<class 'polars.series.series.Series'>


make
cat
"""Alfa Romeo"""
"""Ferrari"""
"""Dodge"""
"""Dodge"""
"""Subaru"""
…
"""Subaru"""
"""Subaru"""
"""Subaru"""


---
* Next, we make a NumPy function that standardizes a column.
* This will be a function that subtracts the mean and divide it by the standard deviation.
* We need to convert the Polars series to NumPy with `.to_numpy()`

In [112]:
def standardize(arr_pl):
    arr = arr_pl.to_numpy()
    return (arr - np.mean(arr)) / np.std(arr)

(
    autos
    .select(city08_standardized = pl.col('city08').map_batches(standardize))
)

city08_standardized
f64
-0.073412
-0.863433
0.242597
-0.784431
-0.231416
…
-0.073412
0.005591
-0.152414


For comparision, we can also do this wholly in Polars.


In [113]:
def standardize_pl(col):
    return (col - col.mean()) / col.std()

(
    autos
    .select(city08_standardized = standardize_pl(pl.col('city08')))
)

city08_standardized
f64
-0.073411
-0.863424
0.242594
-0.784423
-0.231413
…
-0.073411
0.005591
-0.152412


---
Compare the timing of both:

In [114]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
    
    def standardize(arr_pl):
        arr = arr_pl.to_numpy()
        return (arr - np.mean(arr)) / np.std(arr)
    
    (
        autos
        .select(city08_standardized = pl.col('city08').map_batches(standardize))
    )

timing_test()

2.31 ms ± 592 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [115]:
%%timeit
@pl.Config(set_verbose=False)
def timing_test():
           
    def standardize_pl(col):
        return (col - col.mean()) / col.std()
    
    (
        autos
        .select(city08_standardized = standardize_pl(pl.col('city08')))
    )

timing_test()

909 µs ± 196 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
